<a href="https://colab.research.google.com/github/Kanyanjua-Mwangi/Wakulima-Drafts-Football-Draft-Game./blob/main/Wakulima_Drafts_Gameplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Importing Libraries.

In [ ]:
import random
import pandas as pd

#2. Importing Data

In [ ]:
#Import the data.
file_name = '/content/fantasy_points (1).csv'
fantasy_points_df = pd.read_csv(file_name)

#3.0 Gameplay.

In [ ]:
# 🚀 Ask how many players will participate (up to 5)
while True:
    try:
        NUM_PLAYERS = int(input("Enter number of players (1 to 5): "))
        if 1 <= NUM_PLAYERS <= 5:
            break
        else:
            print("❌ Please enter a number between 1 and 5.")
    except ValueError:
        print("❌ Invalid input! Please enter a valid number.")

players = [f"Player {i+1}" for i in range(NUM_PLAYERS)]

#Define the formations available to the users.

FORMATIONS = {
    "4-3-3": {"gk": 1, "df": 4, "mf": 3, "fw": 3},
    "4-4-2": {"gk": 1, "df": 4, "mf": 4, "fw": 2},
    "3-5-2": {"gk": 1, "df": 3, "mf": 5, "fw": 2},
    "3-4-3": {"gk": 1, "df": 3, "mf": 4, "fw": 3},
    "4-5-1": {"gk": 1, "df": 4, "mf": 5, "fw": 1},
    "5-3-2": {"gk": 1, "df": 5, "mf": 3, "fw": 2},
    "5-4-1": {"gk": 1, "df": 5, "mf": 4, "fw": 1}
}

#Define the function for users to select formation.

def get_user_formation(player):
    print(f"\n{player}, select your formation:")
    for idx, formation in enumerate(FORMATIONS.keys(), 1):
        print(f"{idx}. {formation}")

    while True:
        try:
            choice = int(input("\nSelect a formation by number: "))
            if 1 <= choice <= len(FORMATIONS):
                return list(FORMATIONS.keys())[choice - 1]
            else:
                print("Invalid choice! Please select a valid number.")
        except ValueError:
            print("Invalid input! Enter a number corresponding to a formation.")

#Define function to initialize selected formations.
def initialize_squads(players):
    squads = {}
    for player in players:
        formation = get_user_formation(player)
        squads[player] = {"formation": formation, "squad": {pos: [] for pos in FORMATIONS[formation]}}
    return squads

def draft_player(player_name, position, season, player_pool, squads, current_player):
    # Filter by player name, season, and position
    player = player_pool[
        (player_pool["Player"] == player_name) &
        (player_pool["Season"] == season) &
        (player_pool["Position"] == position)
    ]
    if player.empty:
        print(f"{player_name} ({season}, {position}) is not available. Choose another player, season, or position.")
        return False

    user_formation = squads[current_player]["formation"]
    pos_key = position.lower()

    if pos_key not in FORMATIONS[user_formation]:
        print(f"Position '{pos_key}' is not in your formation. Choose another position.")
        return False

    if len(squads[current_player]["squad"][pos_key]) >= FORMATIONS[user_formation][pos_key]:
        print(f"{pos_key.upper()} position is already full in your squad. Choose another position.")
        return False

    squads[current_player]["squad"][pos_key].append(player.iloc[0].to_dict())
    player_pool.drop(player.index, inplace=True)
    return True

# Assume fantasy_points_df is loaded already as player_pool
player_pool = fantasy_points_df.copy()
squads = initialize_squads(players)
random.shuffle(players)
print("\nDraft Order:", " → ".join(players))

while any(
    len(squads[p]["squad"][pos]) < FORMATIONS[squads[p]["formation"]][pos]
    for p in players for pos in FORMATIONS[squads[p]["formation"]]
):
    for current_player in players:
        if all(len(squads[current_player]["squad"][pos]) >= FORMATIONS[squads[current_player]["formation"]][pos]
               for pos in FORMATIONS[squads[current_player]["formation"]]):
            continue

        print(f"\n{current_player}, it's your turn!")
        print("\nAvailable Players:")
        print(player_pool[["Player", "Position", "Season", "Fantasy Points"]].to_string(index=False))

        while True:
            player_name = input(f"{current_player}, enter the player you want to draft (by name): ")
            possible_seasons = player_pool[player_pool["Player"] == player_name]["Season"].unique()

            if len(possible_seasons) == 0:
                print("Invalid player name or already drafted. Please try again.")
                continue

            # Select season if multiple available
            if len(possible_seasons) > 1:
                print(f"{player_name} is available for seasons:")
                for i, s in enumerate(possible_seasons, 1):
                    print(f"{i}. {s}")
                while True:
                    try:
                        season_choice = int(input("Select season by number: "))
                        if 1 <= season_choice <= len(possible_seasons):
                            season_selected = possible_seasons[season_choice - 1]
                            break
                        else:
                            print("Invalid choice. Try again.")
                    except ValueError:
                        print("Please enter a valid number.")
            else:
                season_selected = possible_seasons[0]
                print(f"Only one season available: {season_selected}")

            # Get all positions for this player in the selected season
            possible_positions = player_pool[
                (player_pool["Player"] == player_name) & (player_pool["Season"] == season_selected)
            ]["Position"].unique()

            # If multiple positions, ask which position user wants to assign
            if len(possible_positions) > 1:
                print(f"{player_name} can play multiple positions:")
                for i, pos in enumerate(possible_positions, 1):
                    print(f"{i}. {pos}")
                while True:
                    try:
                        pos_choice = int(input("Select position by number: "))
                        if 1 <= pos_choice <= len(possible_positions):
                            position_selected = possible_positions[pos_choice - 1]
                            break
                        else:
                            print("Invalid choice. Try again.")
                    except ValueError:
                        print("Please enter a valid number.")
            else:
                position_selected = possible_positions[0]
                print(f"Only one position available: {position_selected}")

            # Attempt to draft player with chosen season and position
            if draft_player(player_name, position_selected, season_selected, player_pool, squads, current_player):
                break

print("\nFinal Squads:")
for user, details in squads.items():
    print(f"\n{user}'s Squad ({details['formation']} Formation):")
    for pos, players_list in details["squad"].items():
        for pinfo in players_list:
            print(f"- {pinfo['Player']} ({pos.upper()}) - Season: {pinfo['Season']} - Fantasy Points: {pinfo['Fantasy Points']}")

Streaming output truncated to the last 5000 lines.
    Dominic Calvert-Lewin       fw  22/23          105.63
             Diego Carlos       df  22/23           42.14
           Fabio Carvalho       fw  22/23           41.28
           Fabio Carvalho       mf  22/23           43.87
                 Casemiro       mf  22/23           87.30
             Kaelan Casey       df  22/23           -1.00
               Matty Cash       df  22/23          139.49
             Eiran Cashin       df  22/23            0.34
         Timothy Castagne       df  22/23          104.86
          Trevoh Chalobah       df  22/23           97.11
          Trevoh Chalobah       df  22/23           59.44
            Conor Chaplin       mf  22/23           58.65
            Conor Chaplin       fw  22/23           46.94
         Youssef Chermiti       fw  22/23            0.61
         Youssef Chermiti       mf  22/23            0.98
          Federico Chiesa       fw  22/23            5.29
             Ben Chil

In [ ]:
# **Final Squads with Total Fantasy Points**
print("\nFinal Squads:")

for user, details in squads.items():
    print(f"\n{user}'s Squad ({details['formation']} Formation):")
    total_points = 0  # Initialize total points for the squad

    for pos, players_list in details["squad"].items():
        for player_info in players_list:
            name = player_info['Player']
            season = player_info['Season']
            points = player_info['Fantasy Points']
            total_points += points  # Accumulate total points

            print(f"- {name} ({pos}) - Season: {season} - Fantasy Points: {points}")

    print(f"⭐ Total Fantasy Points: {round(total_points, 2)}")


Final Squads:

Player 1's Squad (4-3-3 Formation):
- José Sá (gk) - Season: 23/24 - Fantasy Points: 212.75
- Kieran Trippier (df) - Season: 24/25 - Fantasy Points: 107.99
- Trent Alexander-Arnold (df) - Season: 23/24 - Fantasy Points: 221.1
- Ola Aina (df) - Season: 24/25 - Fantasy Points: 209.7
- John Stones (df) - Season: 21/22 - Fantasy Points: 80.53
- Joshua Zirkzee (mf) - Season: 24/25 - Fantasy Points: 102.98
- Ismaila Sarr (mf) - Season: 24/25 - Fantasy Points: 275.3
- Cole Palmer (mf) - Season: 23/24 - Fantasy Points: 414.32
- Willian (fw) - Season: 23/24 - Fantasy Points: 143.16
- Teemu Pukki (fw) - Season: 21/22 - Fantasy Points: 239.18
- Bukayo Saka (fw) - Season: 23/24 - Fantasy Points: 332.52
⭐ Total Fantasy Points: 2339.53

Player 2's Squad (4-4-2 Formation):
- Matz Sels (gk) - Season: 24/25 - Fantasy Points: 222.85
- Kyle Walker (df) - Season: 22/23 - Fantasy Points: 51.76
- Leny Yoro (df) - Season: 22/23 - Fantasy Points: 59.48
- Kieran Trippier (df) - Season: 23/24 - 

In [2]:
# Create README.md
with open("README.md", "w") as f:
    f.write("""# Wakulima Drafts Gameplay

Wakulima Drafts is an interactive fantasy football draft game built using Python and Jupyter Notebook, designed for up to 5 players. Each participant selects a tactical formation and drafts players in turns based on real-world stats and fantasy points.

## Features

- Supports 1–5 human players
- Multiple tactical formations (e.g., 4-3-3, 4-4-2, 3-5-2, etc.)
- Turn-based drafting from a player pool with position and season logic
- Real-time squad validation against formation rules

## Requirements

- `pandas`
- A cleaned player dataset named `fantasy_points_df` with columns:
  - `Player`
  - `Position`
  - `Season`
  - `Fantasy Points`

## How to Use

Run the notebook in Google Colab or Jupyter, load your player dataset, and follow the prompts to draft your squad.

## License

This project is licensed under the MIT License.
""")

# Create .gitignore
with open(".gitignore", "w") as f:
    f.write(""".ipynb_checkpoints/
*.ipynb-output.json
__pycache__/
*.py[cod]
*.so
.env
*.env
venv/
env/
*.egg-info/
.Python
pip-log.txt
.DS_Store
Thumbs.db
*.csv
*.xlsx
*.parquet
*.pkl
*.h5
token.txt
credentials.json
""")

# Create LICENSE
with open("LICENSE", "w") as f:
    f.write("""MIT License

Copyright (c) 2025 Samuel

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
""")
